## Get the documents

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## 1.1 Define Elastic Search client

In [42]:
from elasticsearch import Elasticsearch

In [43]:
es_client = Elasticsearch('http://localhost:9200')

## 1.2 Preparing index settings for Elastic Search

In [44]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'intro-homework'

In [45]:
es_client.indices.delete(index=index_name, ignore_unavailable = True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'intro-homework'})

## 1.3 Indexing the data

In [46]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [47]:
query = "How do I execute a command in a running docker container?"

## 1.4 Searching by key-word (not vector search)

In [48]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [51]:
response = es_client.search(index = index_name, body = search_query)

In [53]:
response

ObjectApiResponse({'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'intro-homework', '_id': 'qRm-z5ABI_SuyNcymFLz', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'intro-homework', '_id': 'txm-z5ABI_SuyNcylVC9', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker conta

In [54]:
response['hits']['hits'][0]['_score']

84.050095

## 1.5 Filtering in the query

In [17]:
search_query_filtering = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response = es_client.search(index = index_name, body = search_query_filtering)
response['hits']['hits']

[{'_index': 'intro-homework',
  '_id': '9RlEz5ABI_SuyNcyWU6H',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'intro-homework',
  '_id': 'FBlEz5ABI_SuyNcyWU_J',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_d

## 2.1 Building a prompt to send to the LLM

In [29]:
context_template = ''

In [30]:
for doc in response['hits']['hits']:
    context_template = context_template + f'Q: {doc['_source']['question']}\nA: {doc['_source']['text']}\n\n'

In [31]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [32]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [33]:
prompt = prompt_template.format(question = query, context = context_template)

In [34]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [35]:
print(len(prompt.strip()))

1462


## 2.2 Calculate the number of tokens in our query

In [37]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [38]:
len(encoding.encode(prompt.strip()))

322

In [39]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [40]:
from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
model = 'gpt-4o',
messages = [{'role' : 'user', 'content' : prompt}])
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container using `docker ps`, then use the `docker exec` command with the `-it` flag to run your desired command inside the container. Here's how you can do it:

1. List running containers to find the container ID:
   ```sh
   docker ps
   ```

2. Execute a command in the specific container (e.g., open a bash session):
   ```sh
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your container.
